In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import os
import shutil
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

/tmp/ipykernel_2097/2746632847.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was too old on your system - pyarrow 10.0.1 is the current minimum supported version as of this release.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
/home/maha/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-02-28 11:59:08.894534: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off

In [2]:
df = pd.read_csv('raw_data/Data.csv')
df

,site_id,patient_id,image_id,laterality,view,age,cancer,biopsy,invasive,BIRADS,implant,density,machine_id,difficult_negative_case
0,2,10006,462822612,L,CC,61.0,0,0,0,NaN,0,NaN,29,False
1,2,10006,1459541791,L,MLO,61.0,0,0,0,NaN,0,NaN,29,False
2,2,10006,1864590858,R,MLO,61.0,0,0,0,NaN,0,NaN,29,False
3,2,10006,1874946579,R,CC,61.0,0,0,0,NaN,0,NaN,29,False
4,2,10011,220375232,L,CC,55.0,0,0,0,0.0,0,NaN,21,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54701,1,9973,1729524723,R,MLO,43.0,0,0,0,1.0,0,C,49,False
54702,1,9989,63473691,L,MLO,60.0,0,0,0,NaN,0,C,216,False
54703,1,9989,1078943060,L,CC,60.0,0,0,0,NaN,0,C,216,False
54704,1,9989,398038886,R,MLO,60.0,0,0,0,0.0,0,C,216,True


In [3]:
new_df = df.drop(['site_id', 'laterality','view','biopsy','BIRADS','implant','density','machine_id','difficult_negative_case','invasive'], axis=1)
new_df

,patient_id,image_id,age,cancer
0,10006,462822612,61.0,0
1,10006,1459541791,61.0,0
2,10006,1864590858,61.0,0
3,10006,1874946579,61.0,0
4,10011,220375232,55.0,0
...,...,...,...,...
54701,9973,1729524723,43.0,0
54702,9989,63473691,60.0,0
54703,9989,1078943060,60.0,0
54704,9989,398038886,60.0,0


In [4]:
cancer_negative = new_df[new_df['cancer'] == 0]
cancer_positive = new_df[new_df['cancer'] == 1]

In [5]:
cancer_positive

,patient_id,image_id,age,cancer
87,10130,388811999,71.0,1
88,10130,613462606,71.0,1
89,10130,1360338805,71.0,1
90,10130,1672636630,71.0,1
190,10226,461614796,71.0,1
...,...,...,...,...
54485,9707,1900919592,56.0,1
54593,9840,111786935,59.0,1
54594,9840,1559360810,59.0,1
54595,9851,1409270544,58.0,1


In [6]:
cancer_negative

,patient_id,image_id,age,cancer
0,10006,462822612,61.0,0
1,10006,1459541791,61.0,0
2,10006,1864590858,61.0,0
3,10006,1874946579,61.0,0
4,10011,220375232,55.0,0
...,...,...,...,...
54701,9973,1729524723,43.0,0
54702,9989,63473691,60.0,0
54703,9989,1078943060,60.0,0
54704,9989,398038886,60.0,0


In [7]:
source_directory = "./raw_data/images_512px8bit/train_images"
destination_directory = "./raw_data/positive"
for index, row in cancer_positive.iterrows():
    # Get the image filename from the 'image_id' column
    image_filename = row['image_id']
    patient_filename = row['patient_id']
    # Construct the source path
    source_path = source_directory + '/' + str(patient_filename).split('.')[0] + '/' + str(image_filename).split('.')[0]+'.png'
    # Construct the destination path
    destination_path = destination_directory +'/'+  str(image_filename) + '.png'
    # Copy the image file from source to destination
    shutil.copy(source_path, destination_path)

In [8]:
source_directory = "./raw_data/images_512px8bit/train_images"
destination_directory = "./raw_data/negative"
for index, row in cancer_negative.iterrows():
    # Get the image filename from the 'image_id' column
    image_filename = row['image_id']
    patient_filename = row['patient_id']
    # Construct the source path
    source_path = source_directory + '/' + str(patient_filename).split('.')[0] + '/' + str(image_filename).split('.')[0]+'.png'
    # Construct the destination path
    destination_path = destination_directory +'/'+  str(image_filename) + '.png'
    # Copy the image file from source to destination
    shutil.copy(source_path, destination_path)

In [9]:
class_counts = df['cancer'].value_counts()
class_counts

cancer
0    53548
1     1158
Name: count, dtype: int64

In [10]:
target_samples_per_class = 1150

balanced_data = []
for cls in class_counts.index:
    samples_needed = target_samples_per_class - class_counts[cls]
    if samples_needed <= 0:
        samples = new_df[new_df['cancer'] == cls].sample(target_samples_per_class)
    else:
        # Duplicate rows to meet the target_samples_per_class
        samples = pd.concat([new_df[new_df['cancer'] == cls]] * (samples_needed // class_counts[cls] + 1), axis=0, ignore_index=True)[:target_samples_per_class]
    balanced_data.append(samples)

balanced_data = pd.concat(balanced_data)

balanced_data.shape

(2300, 4)

In [11]:
balanced_data

,patient_id,image_id,age,cancer
20552,31789,2141081134,76.0,0
29214,41176,338696633,40.0,0
26072,37855,257412882,58.0,0
45765,59276,747713950,51.0,0
9881,20578,1946247399,55.0,0
...,...,...,...,...
215,1025,773597682,56.0,1
49144,63053,1562088774,54.0,1
8989,1963,159974570,67.0,1
37863,50644,1779556127,64.0,1


In [12]:
img_width, img_height = 224, 224
batch_size = 10


dataset_dir = "./raw_data"
X = []
y = balanced_data['cancer']  # Assuming 'target' is the column representing the labels

for _, row in balanced_data.iterrows():
    image_id = row['image_id']
    image_path = os.path.join(dataset_dir, 'positive' if row['cancer'] == 1 else 'negative', f'{image_id}.png')
   
    # Check if the image file exists
    if os.path.exists(image_path):
        image = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_width, img_height))
        image = tf.keras.preprocessing.image.img_to_array(image)
        X.append(image)

X = np.array(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
